In [2]:
# import clip_fdt as fdt
# from clip_fdt import Query_model, Clip_FDT, AllGather
# from loss import ClipInfoCELoss
import torch
from torch import nn

In [8]:
dir(fdt)

NameError: name 'fdt' is not defined

# 现状

手里有模态的token，如何构建一个字典，将这些token重建，并进行对比学习？

In [9]:
# 参数设置
# 字典
sd_num = 16384
sd_dim = 512
space_dict = torch.randn(sd_num, sd_dim)
sd_temperature = 2.0 # Softmax温度系数
att_func_type = 'sparsemax'
pool_type = 'max'
raw_img_ft_dim = raw_txt_ft_dim = raw_acoustic_ft_dim = 50

NameError: name 'torch' is not defined

In [10]:
# 解耦的同质特征 common feature 
# torch.Size([16, 50, 46]) torch.Size([16, 50, 925]) torch.Size([16, 50, 232])
# [batch_size, feature_dim, seq_len]
com_language = torch.randn(16, 50, 46)
com_audio = torch.randn(16, 50, 925)
com_vision = torch.randn(16, 50, 232)

com_language = com_language.transpose(1, 2)
com_audio = com_audio.transpose(1, 2)
com_vision = com_vision.transpose(1, 2)

NameError: name 'torch' is not defined

Query_model会做两件事：
- 使用MLP将输入模态的特征维度 feature_dim 转为字典特征维度 sd_dim
- 返回注意力权重，重构特征，字典

In [ ]:
img_query_model = Query_model(ft_dim=raw_img_ft_dim,
                              sd_dim=sd_dim,
                              temperature=sd_temperature,
                              att_func_type=att_func_type,
                              pool_type=pool_type)

In [ ]:
# 提取图像的基于字典的重构特征
att_weight, sd_img_ft, img_k = img_query_model(com_language, space_dict)

In [ ]:
print(type(att_weight))
print(type(sd_img_ft))
# 字典
print(type(img_k))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [ ]:
print(f"The shape of att_weight: {att_weight.shape}")
print(f"The shape of sd_img_ft: {sd_img_ft.shape}")
print(f"The shape of img_k: {img_k.shape}")

The shape of att_weight: torch.Size([16, 16384])
The shape of sd_img_ft: torch.Size([16, 512])
The shape of img_k: torch.Size([16384, 512])


In [ ]:
att_weight[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SelectBackward>)

In [ ]:
sd_img_ft[0].shape

torch.Size([512])

In [ ]:
clip_fdt = Clip_FDT(
    image_encode=None,
    text_encode=None,
    use_allgather=None,
    sd_num=sd_num,
    sd_dim=sd_dim,
    raw_img_ft_dim=raw_img_ft_dim,
    raw_txt_ft_dim=raw_txt_ft_dim,
    raw_acoustic_ft_dim=raw_acoustic_ft_dim,
    att_func_type=att_func_type,
    pool_type=pool_type,
    sd_temperature=sd_temperature
)

In [ ]:
x = clip_fdt(com_language, com_audio, com_vision)

In [ ]:
logits_per_image_sd_LV, logits_per_text_sd_LV = x[0]

In [ ]:
logits_per_image_sd_LV.shape

torch.Size([16, 16])

In [ ]:
logits_per_text_sd_LV.shape

torch.Size([16, 16])

In [ ]:
clip_loss = ClipInfoCELoss()

In [ ]:
clip_loss(logits_per_image_sd_LV, logits_per_text_sd_LV)

(tensor(2.8579, grad_fn=<DivBackward0>),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]))

In [ ]:
torch.arange(16)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [ ]:
(1, 2), \
(3, 4)

((1, 2), (3, 4))

In [ ]:
_, sd_acoustic_ft, _ = clip_fdt.extract_acoustic_sd_ft(com_audio)

In [ ]:
sd_acoustic_ft.shape

torch.Size([16, 512])

In [2]:
import torch

/opt/conda/envs/mmsa/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
st = []
a = torch.rand(16, 1)
b = torch.rand(16, 1)

In [4]:
st.append(a)
st.append(b)

In [5]:
torch.stack(st).shape

torch.Size([2, 16, 1])